In [1]:
import pandas as pd
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
csv_file = pd.read_csv(f"{location}/ERC_correlation.csv")

csv_file.index = csv_file.columns
string_to_remove = "_nostop.fas.aligned.fas.best"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)

string_to_remove = ".fas.ClipKIT"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)


matrix_long = csv_file.reset_index().melt(
    id_vars="index",         # Keeps row names as a column
    var_name="Gene1",       # New name for columns
    value_name="Gene2"       # New name for matrix values
)

# Rename 'index' to 'Row' for clarity
diapause = matrix_long.rename(columns={"index": "Row"})

location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"

# diapause = pd.read_csv(f"{location}/ERC_correlation_diapause_long.csv")
# location = "/mnt/c/Users/Saurav Baral/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
gene_anno_dic = {}
with open(f"{location}/correlation_2_with_annotation.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        gene_name = lines.split(",")[0].split("_")[0][1:]
        anno_name = lines.split(",")[-1].replace('"', '')
        gene_anno_dic[gene_name]= anno_name.strip()


diapause_rn = diapause.rename(columns={"Gene2": "Correlation"})
diapause_rn = diapause_rn.rename(columns={"Gene1": "Gene2"})
diapause_rn = diapause_rn.rename(columns={"Row": "Gene1"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

diapause_rn["Annotation_gene1"] = diapause_rn["Gene1"].apply(check_annotation)
diapause_rn["Annotation_gene2"] = diapause_rn["Gene2"].apply(check_annotation)

diapause_rn["Correlation"].fillna(1, inplace=True)

# print(diapause_rn)

genes = diapause_rn["Gene1"].unique()
# df_filtered = diapause_rn[diapause_rn["Correlation"] >= 0.1]
# diapause_rn["Correlation"] += 1
df_filtered = diapause_rn
gene_dictionary = {}
to_remove = ["tim","9062at7088","per","TIMELESS","CLOCK"]
for row in df_filtered.itertuples():
    if (row.Annotation_gene1 not in to_remove) and (row.Annotation_gene2 not in to_remove):
        if row.Annotation_gene1 not in gene_dictionary:
            gene_dictionary[row.Annotation_gene1] = {}
        if row.Annotation_gene1 != row.Annotation_gene2:
            gene_dictionary[row.Annotation_gene1][row.Annotation_gene2] = row.Correlation

gene_dict = gene_dictionary
print("done")

import networkx as nx
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform

genes = list(gene_dict.keys())

# Initialize a similarity matrix
n = len(genes)
similarity_matrix = np.zeros((n, n))



for i, gene1 in enumerate(genes):
    for j, gene2 in enumerate(genes):
        if gene2 in gene_dict[gene1]:
            similarity_matrix[i, j] = round(gene_dict[gene1][gene2],2)
            similarity_matrix[j, i] = round(gene_dict[gene1][gene2],2)  # Ensure symmetry
        else:
            similarity_matrix[i, j] = 0  # No direct similarity (or self-similarity)
            similarity_matrix[j, i] = 0


min_similarity = np.min(similarity_matrix)
print(min_similarity)

# Convert similarity matrix to a distance matrix
distance_matrix = 1 - similarity_matrix

np.fill_diagonal(distance_matrix, 0)

# Flatten the distance matrix to a condensed form (required by scipy)
condensed_distance_matrix = squareform(distance_matrix)


done
-0.58


In [2]:
import scipy.cluster.hierarchy as sch
from collections import defaultdict

Z_original = sch.linkage(condensed_distance_matrix, method="average")

# Bootstrapping
num_bootstraps = 100  # Number of bootstrap resamples
bootstrap_support = defaultdict(int)

for num in range(num_bootstraps):
    # Resample genes with replacement
    print(num)
    resampled_indices = np.random.choice(len(genes), size=len(genes), replace=True)
    resampled_genes = [genes[i] for i in resampled_indices]
    
    # Subset distance matrix
    resampled_distances = distance_matrix[np.ix_(resampled_indices, resampled_indices)]
    
    # Compute linkage on bootstrap sample
    condensed_resampled = squareform(resampled_distances)
    Z_resampled = sch.linkage(condensed_resampled, method="average")
    
    # Convert to set of clusters
    for i in range(len(Z_resampled)):
        cluster = frozenset(resampled_genes[int(Z_resampled[i, 0]) : int(Z_resampled[i, 1]) + 1])
        bootstrap_support[cluster] += 1

# Compute bootstrap percentages
for cluster in bootstrap_support:
    bootstrap_support[cluster] = (bootstrap_support[cluster] / num_bootstraps) * 100

# Function to create Newick format with bootstrap support
def generate_newick(Z, bootstrap_support, genes):
    def recurse(node):
        if node < len(genes):  # Leaf node
            return genes[int(node)]
        else:  # Internal node
            left = recurse(Z[int(node - len(genes)), 0])
            right = recurse(Z[int(node - len(genes)), 1])
            support = bootstrap_support.get(frozenset([left, right]), 0)
            return f"({left},{right}){':' + str(round(support, 2)) if support > 0 else ''}"

    # Start the recursive generation of the Newick string
    return recurse(len(Z) + len(genes) - 1) + ';'

# Generate Newick tree with bootstrap support
newick_tree = generate_newick(Z_original, bootstrap_support, genes)

# Print the Newick tree with bootstrap values
print("Newick Tree with Bootstrap Confidence:")
print(newick_tree)


0
1
Newick Tree with Bootstrap Confidence:
((((((VPS33A,(GNS,4668at7088)),((IFT122,RILPL1),(DNA2,(SNX2,8779at7088)))),((((PRPF40A,ATG9A),((UBXN6,NUP155),(SH3GL1,RB1CC1))),((SRRT,(KIFAP3,VPS8)),((PPFIBP2,CCDC39),(2325at7088,(CIT,REPS1))))),((((TXNDC11,NCSTN),((GLIS2,8186at7088),(WDR81,ATR))),(((550at7088,(1037at7088,8408at7088)),(RAB11FIP4,(1248at7088,rom-4))),(CAP1,(1391at7088,ABHD2)))),((CLASRP,UBE3C),((FNIP1,KIAA1468),((FOXN4,NPRL3),(4492at7088,(SART3,GOLGA1)))))))),(((11910at7088,TCHP),(((PAIP1,Mct1),((ZGPAT,WAPAL),(CD109,(HOXA3,SMG6)))),((2442at7088,(OGFOD1,(SPIRE2,NELL2))),(3807at7088,(TAF6,993at7088))))),((1999at7088,(SLC4A3,CLIPB9)),((SMOC1,2109at7088),((13072at7088,5987at7088),(VAPA,RN-tre)))))),((((15011at7088,7285at7088),(MEPCE,GGCX)),(6646at7088,(NECAP1,(1418at7088,12626at7088)))),((((((CCBE1,(4731at7088,DPP10)),(SPATA20,(SLC33A1,PRDX4))),(((TMEM129,HEATR3),((10333at7088,CHID1),(PNLDC1,(ZNFX1,GATB)))),(((1136at7088,CEP131),(PGK1,9243at7088)),((TIMMDC1,(PDCD5,5239at7088)),((9

In [ ]:
import scipy.cluster.hierarchy as sch
from collections import defaultdict
import numpy as np
import gc
num_bootstraps = 10
Z_original = sch.linkage(condensed_distance_matrix, method="average")

def generate_bootstrap_support(num_bootstraps, genes, distance_matrix):
    bootstrap_support = defaultdict(int)
    for num in range(num_bootstraps):
        resampled_indices = np.random.choice(len(genes), size=len(genes), replace=True)
        resampled_genes = [genes[i] for i in resampled_indices]
        resampled_distances = distance_matrix[np.ix_(resampled_indices, resampled_indices)]
        condensed_resampled = sch.distance.squareform(resampled_distances)
        Z_resampled = sch.linkage(condensed_resampled, method="average")
        for i in range(len(Z_resampled)):
            cluster = frozenset(resampled_genes[int(Z_resampled[i, 0]):int(Z_resampled[i, 1]) + 1])
            bootstrap_support[cluster] += 1
        del resampled_indices, resampled_genes, resampled_distances, condensed_resampled, Z_resampled
        gc.collect()
    return bootstrap_support

# Generate bootstrap support
bootstrap_support = generate_bootstrap_support(num_bootstraps, genes, distance_matrix)

# Compute bootstrap percentages
for cluster in bootstrap_support:
    bootstrap_support[cluster] = (bootstrap_support[cluster] / num_bootstraps) * 100

# Function to create Newick format with bootstrap support
def generate_newick(Z, bootstrap_support, genes):
    def recurse(node):
        if node < len(genes):  # Leaf node
            return genes[int(node)]
        else:  # Internal node
            left = recurse(Z[int(node - len(genes)), 0])
            right = recurse(Z[int(node - len(genes)), 1])
            support = bootstrap_support.get(frozenset([left, right]), 0)
            return f"({left},{right}){':' + str(round(support, 2)) if support > 0 else ''}"

    # Start the recursive generation of the Newick string
    return recurse(len(Z) + len(genes) - 1) + ';'

# Generate Newick tree with bootstrap support
newick_tree = generate_newick(Z_original, bootstrap_support, genes)

# Print the Newick tree with bootstrap values
print("Newick Tree with Bootstrap Confidence:")
print(newick_tree)